In [16]:
# This is necessary for jupyter to draw graphs offline.
# Plotly tries to use cloud service to draw graphs by default.
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

<script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script><script type="text/javascript">if (window.MathJax) {MathJax.Hub.Config({SVG: {font: "STIX-Web"}});}</script><script>requirejs.config({paths: { 'plotly': ['https://cdn.plot.ly/plotly-latest.min']},});if(!window._Plotly) {require(['plotly'],function(plotly) {window._Plotly=plotly;});}</script>

In [17]:
import GPy
import optuna
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.io as pio
    

seed = 123
np.random.seed(seed=seed)

In [18]:
variance = 1.
lengthscale = 0.2
quantization = 50
epsilon = 0.1
sigma2 = 0.01

In [19]:
kernel = GPy.kern.RBF(input_dim=2, variance=variance, lengthscale=lengthscale)

x = np.linspace(0., 1., quantization)
xx = [[0., 0.] for _ in range(quantization ** 2)]
for i in range(quantization):
    for j in range(quantization):
        xx[i * quantization + j][0] = x[i]
        xx[i * quantization + j][1] = x[j]
xx = np.asarray(xx)
mu_rbf = np.zeros(quantization ** 2)
cov_rbf = kernel.K(xx, xx) + sigma2 * np.eye(quantization ** 2)

y_rbf = np.zeros(quantization ** 2)
y_rbf_best = []


def quantize(a):
    return np.argmin(np.abs(np.linspace(0., 1., quantization) - a))


def fun_rbf(a, b):
    global y_rbf
    global y_rbf_best
    y_rbf = np.sqrt(1. - epsilon) * y_rbf + np.sqrt(epsilon) * np.random.multivariate_normal(mu_rbf, cov_rbf)
    y_rbf_best.append(np.max(y_rbf))
    i = quantize(a)
    j = quantize(b)
    return y_rbf[i + quantization + j]


def objevtive_rbf(trial):
    a, b = trial.suggest_joint('ab',
                               [['uniform', 'a', 0., 1.],
                                ['uniform', 'b', 0., 1.]])
    return fun_rbf(a, b)

In [20]:
n_trials = 100

In [ ]:
sampler = optuna.samplers.GPSampler()
study_rbf = optuna.create_study(sampler=sampler)
study_rbf.optimize(lambda t: - objevtive_rbf(t), n_trials=n_trials)

In [15]:
trials = study_rbf.trials
trial_values = [trial.value for trial in trials]

assert len(trial_values) == len(y_rbf_best)

cum_regret = 0.
avg_cum_regrets = []
for t, best_value, trial_value in enumerate(zip(y_rbf_best, trial_values)):
    cum_regret += best_value - trial_value
    avg_cum_regrets.append(cum_regret / (t + 1))

data = [
    go.Scatter(x=np.arange(len(avg_cum_regrets)) + 1, y=avg_cum_regrets, mode='markers')
]
iplot(data, filename='syn_bogunovic_rbf')

NameError: name 'study_rbf' is not defined